In [1]:
import numpy as np
import pandas as pd
import re
train_data = pd.read_json('/Users/vaibhavgupta/gatom/gatom-ml/data/hotstar_dataset/train_data.json',orient='index')


In [2]:
train_data['g1'] = [re.sub(pattern='\:\d+',repl='',string=x) for x in train_data['genres']]
train_data['g1'] = train_data['g1'].apply(lambda x: x.split(','))
t1 = pd.Series(train_data['g1']).apply(frozenset).to_frame(name='t_genre')
genreSet = frozenset.union(*t1.t_genre)

In [3]:
def string_genres_time(genre, string):
 if genre in string:
   prbmStr = string[string.index(genre) :]
   #print(prbmStr)
   problemStr = prbmStr[prbmStr.index(':')+1:]
   #print(problemStr)
   #print(problemStr.index(','))
   if ',' in problemStr:
     return int(problemStr[0:problemStr.index(',')])
   else:
     return int(problemStr[0:])
 else:
   return 0

In [4]:
def string_dow_time(genre, string):
 if genre in string:
   prbmStr = string[string.index(genre) :]
   #print(prbmStr)
   problemStr = prbmStr[prbmStr.index(':')+1:]
   #print(problemStr)
   #print(problemStr.index(','))
   if ',' in problemStr:
     return int(problemStr[0:problemStr.index(',')])
   else:
     return int(problemStr[0:])
 else:
   return 0

In [5]:
def string_tod_time(genre, string):
    if genre in string:
        prbmStr = string[string.index(genre) :]
        #print(prbmStr)
        problemStr = prbmStr[prbmStr.index(':')+1:]
        #print(problemStr)
        #print(problemStr.index(','))
        if ',' in problemStr:
            return int(problemStr[0:problemStr.index(',')])
        else:
            return int(problemStr[0:])
    else:
        todData = genre[1:]
        if todData in string:
            if string.index(todData) == 0:
                prbmStr = string[string.index(todData) :]
                #print(prbmStr)
                problemStr = prbmStr[prbmStr.index(':')+1:]
                #print(problemStr)
                #print(problemStr.index(','))
                if ',' in problemStr:
                    return int(problemStr[0:problemStr.index(',')])
                else:
                    return int(problemStr[0:])
            else:
                return 0
        else:
            return 0

In [6]:
for i in genreSet:
  train_data[i] = train_data["genres"].apply(lambda x : string_genres_time(i, x))

In [7]:
for i in np.arange(1,8):
  str_i = str(i) + ':'
  train_data[i] = train_data["dow"].apply(lambda x : string_dow_time(str_i, x))

In [8]:
for i in np.arange(0,24):
  str_i = ','+str(i)+':'
  colName = 'tod_'+str(i)
  train_data[colName] = train_data["tod"].apply(lambda x : string_genres_time(str_i, x))

In [9]:
train_data.drop(['g1','tod','dow','genres'],inplace=True,axis=1)

In [10]:
train_data.drop('cities',inplace=True,axis=1)

In [11]:
def get_int_value(param):
  return int(param[1:])

In [12]:
def getTimeList(titeString):
  return np.array([get_int_value(item) for item in re.findall('\:\d+',titeString)])

In [13]:
train_data['g2'] = [getTimeList(x) for x in train_data['titles']]
#train_data['g2'] = train_data['g2'].apply(lambda x: x.split(','))
#t1 = pd.Series(train_data['g2']).apply(frozenset).to_frame(name='t_titles')
#titlesSet = frozenset.union(*t1.t_titles)

In [14]:
train_data['title_mean'] = train_data['g2'].apply(np.mean)

/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [15]:
train_data['title_median'] = train_data['g2'].apply(np.median)
train_data['title_sum'] = train_data['g2'].apply(np.sum)


In [16]:
train_data['title_count'] = train_data['g2'].apply(len)

In [17]:
train_data.drop('g2',inplace=True,axis=1)

In [18]:
train_data.drop('titles',axis=1,inplace=True)

In [19]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, train-121672 to train-126328
Data columns (total 71 columns):
segment         200000 non-null object
Reality         200000 non-null int64
Boxing          200000 non-null int64
Horror          200000 non-null int64
Formula1        200000 non-null int64
IndiaVsSa       200000 non-null int64
Drama           200000 non-null int64
Comedy          200000 non-null int64
Swimming        200000 non-null int64
Wildlife        200000 non-null int64
FormulaE        200000 non-null int64
Hockey          200000 non-null int64
TalkShow        200000 non-null int64
NA              200000 non-null int64
Athletics       200000 non-null int64
Awards          200000 non-null int64
Family          200000 non-null int64
Teen            200000 non-null int64
Volleyball      200000 non-null int64
Action          200000 non-null int64
Science         200000 non-null int64
Table Tennis    200000 non-null int64
Romance         200000 non-null int64
Li

In [20]:
all_cols = list(train_data.columns)

In [21]:
all_cols.remove('title_count')
all_cols.remove('segment')

In [22]:
train_data[all_cols] = train_data[all_cols]/60

In [23]:
train_data[all_cols] = train_data[all_cols].fillna(0)

In [24]:
train_data[all_cols] = train_data[all_cols].astype(int)

In [25]:
train_data['segment'].replace('pos',1,inplace=True)
train_data['segment'].replace('neg',0,inplace=True)

In [26]:
train_data.head().T

,train-121672,train-121673,train-121670,train-121671,train-121676
segment,0,0,0,0,1
Reality,0,0,0,0,0
Boxing,0,0,0,0,0
Horror,0,0,0,0,0
Formula1,0,0,0,0,0
IndiaVsSa,0,0,0,0,0
Drama,0,815,347,2,0
Comedy,0,70,0,0,0
Swimming,0,0,0,0,0
Wildlife,1,0,0,0,0


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [28]:
y_label = train_data.pop('segment')

In [29]:
X_train, X_test, y_train, y_test = train_test_split(train_data, y_label, test_size=0.3, random_state=9)

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
seed = 9

In [31]:
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
for train_index,test_index in kf.split(X_train,y_train):
    print('{} of KFold {}'.format(i,kf.n_splits))
    xtr,xvl = X_train.iloc[train_index],X_train.iloc[test_index]
    ytr,yvl = y_train.iloc[train_index],y_train.iloc[test_index]
    
    #model
    lr = LogisticRegression()
    lr.fit(xtr,ytr)
    score = roc_auc_score(yvl,lr.predict(xvl))
    print('ROC AUC score:',score)
    cv_score.append(score)   
    print('Test Accuracy: ',accuracy_score(yvl,lr.predict(xvl)))
    print('Train Accuracy: ',accuracy_score(ytr,lr.predict(xtr)))
    print(confusion_matrix(yvl,lr.predict(xvl)))
    print(classification_report(yvl,lr.predict(xvl)))
    print('------------------------------------------------------------------')
    print('Validation Accuracy ',accuracy_score(y_test,lr.predict(X_test)))
    print(confusion_matrix(y_test,lr.predict(X_test)))
    print(classification_report(y_test,lr.predict(X_test)))
    print('------------------------------------------------------------------')
    i+=1

1 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


ROC AUC score: 0.5072241390024437
Test Accuracy:  0.9219285714285714
Train Accuracy:  0.9218928571428572
[[25776    86]
 [ 2100    38]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25862
           1       0.31      0.02      0.03      2138

   micro avg       0.92      0.92      0.92     28000
   macro avg       0.62      0.51      0.50     28000
weighted avg       0.88      0.92      0.89     28000

------------------------------------------------------------------
Validation Accuracy  0.9219
[[55235   200]
 [ 4486    79]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     55435
           1       0.28      0.02      0.03      4565

   micro avg       0.92      0.92      0.92     60000
   macro avg       0.60      0.51      0.50     60000
weighted avg       0.88      0.92      0.89     60000

------------------------------------------------------------------
2 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


ROC AUC score: 0.5070869425031282
Test Accuracy:  0.9220714285714285
Train Accuracy:  0.9220625
[[25781    81]
 [ 2101    37]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25862
           1       0.31      0.02      0.03      2138

   micro avg       0.92      0.92      0.92     28000
   macro avg       0.62      0.51      0.50     28000
weighted avg       0.88      0.92      0.89     28000

------------------------------------------------------------------
Validation Accuracy  0.9220166666666667
[[55247   188]
 [ 4491    74]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     55435
           1       0.28      0.02      0.03      4565

   micro avg       0.92      0.92      0.92     60000
   macro avg       0.60      0.51      0.50     60000
weighted avg       0.88      0.92      0.89     60000

------------------------------------------------------------------
3 of KFold 5


...

ROC AUC score: 0.5067564121549226
Test Accuracy:  0.9218571428571428
Train Accuracy:  0.9219464285714286
[[25776    86]
 [ 2102    36]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25862
           1       0.30      0.02      0.03      2138

   micro avg       0.92      0.92      0.92     28000
   macro avg       0.61      0.51      0.50     28000
weighted avg       0.88      0.92      0.89     28000

------------------------------------------------------------------
Validation Accuracy  0.9218166666666666
[[55238   197]
 [ 4494    71]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     55435
           1       0.26      0.02      0.03      4565

   micro avg       0.92      0.92      0.92     60000
   macro avg       0.59      0.51      0.49     60000
weighted avg       0.87      0.92      0.89     60000

------------------------------------------------------------------
4 of 

/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


ROC AUC score: 0.506912942039127
Test Accuracy:  0.92175
Train Accuracy:  0.9220892857142857
[[25772    90]
 [ 2101    37]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25862
           1       0.29      0.02      0.03      2138

   micro avg       0.92      0.92      0.92     28000
   macro avg       0.61      0.51      0.50     28000
weighted avg       0.88      0.92      0.89     28000

------------------------------------------------------------------
Validation Accuracy  0.92175
[[55234   201]
 [ 4494    71]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     55435
           1       0.26      0.02      0.03      4565

   micro avg       0.92      0.92      0.92     60000
   macro avg       0.59      0.51      0.49     60000
weighted avg       0.87      0.92      0.89     60000

------------------------------------------------------------------
5 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


ROC AUC score: 0.5086079861601178
Test Accuracy:  0.9221071428571429
Train Accuracy:  0.9220357142857143
[[25775    87]
 [ 2094    44]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     25862
           1       0.34      0.02      0.04      2138

   micro avg       0.92      0.92      0.92     28000
   macro avg       0.63      0.51      0.50     28000
weighted avg       0.88      0.92      0.89     28000

------------------------------------------------------------------
Validation Accuracy  0.9218666666666666
[[55237   198]
 [ 4490    75]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     55435
           1       0.27      0.02      0.03      4565

   micro avg       0.92      0.92      0.92     60000
   macro avg       0.60      0.51      0.50     60000
weighted avg       0.88      0.92      0.89     60000

------------------------------------------------------------------


In [33]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_sample(X_train, y_train)

In [34]:
X_train.shape

(140000, 70)

In [35]:
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
for train_index,test_index in kf.split(X_resampled,y_resampled):
    print('{} of KFold {}'.format(i,kf.n_splits))
    xtr,xvl = X_resampled[train_index],X_resampled[test_index]
    ytr,yvl = y_resampled[train_index],y_resampled[test_index]
    
    #model
    lr = LogisticRegression()
    lr.fit(xtr,ytr)
    score = roc_auc_score(yvl,lr.predict(xvl))
    print('ROC AUC score:',score)
    cv_score.append(score)   
    print('Test Accuracy: ',accuracy_score(yvl,lr.predict(xvl)))
    print('Train Accuracy: ',accuracy_score(ytr,lr.predict(xtr)))
    print('Confusion Matrix: ',confusion_matrix(yvl,lr.predict(xvl)))
    print('Validation Accuracy ',accuracy_score(y_test,lr.predict(X_test)))
    print('Validation Confusion Matrix: ',confusion_matrix(y_test,lr.predict(X_test)))
    print('------------------------------------------------------------------')
    i+=1

1 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ROC AUC score: 0.714117237645967
Test Accuracy:  0.714117237645967
Train Accuracy:  0.7116957311886165
Confusion Matrix:  [[19598  6264]
 [ 8523 17339]]
Validation Accuracy  0.7500833333333333
Validation Confusion Matrix:  [[42028 13407]
 [ 1588  2977]]
------------------------------------------------------------------
2 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ROC AUC score: 0.7127832340886242
Test Accuracy:  0.7127832340886242
Train Accuracy:  0.711270396721058
Confusion Matrix:  [[19567  6295]
 [ 8561 17301]]
Validation Accuracy  0.7513666666666666
Validation Confusion Matrix:  [[42099 13336]
 [ 1582  2983]]
------------------------------------------------------------------
3 of KFold 5


...

ROC AUC score: 0.7117005645348388
Test Accuracy:  0.7117005645348388
Train Accuracy:  0.7118117314979506
Confusion Matrix:  [[19684  6178]
 [ 8734 17128]]
Validation Accuracy  0.7527333333333334
Validation Confusion Matrix:  [[42197 13238]
 [ 1598  2967]]
------------------------------------------------------------------
4 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ROC AUC score: 0.7102892274379399
Test Accuracy:  0.7102892274379399
Train Accuracy:  0.7122998994663986
Confusion Matrix:  [[19699  6163]
 [ 8822 17040]]
Validation Accuracy  0.7543
Validation Confusion Matrix:  [[42294 13141]
 [ 1601  2964]]
------------------------------------------------------------------
5 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ROC AUC score: 0.7093225581934884
Test Accuracy:  0.7093225581934886
Train Accuracy:  0.7120775655401748
Confusion Matrix:  [[19587  6275]
 [ 8760 17102]]
Validation Accuracy  0.7515166666666667
Validation Confusion Matrix:  [[42108 13327]
 [ 1582  2983]]
------------------------------------------------------------------


Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [38]:
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
for j in np.arange(1,50,5):
  for train_index,test_index in kf.split(X_resampled,y_resampled):
      print('{} and {} of KFold {}'.format(i,j,kf.n_splits))
      xtr,xvl = X_resampled[train_index],X_resampled[test_index]
      ytr,yvl = y_resampled[train_index],y_resampled[test_index]

      #model
      lr = RandomForestClassifier(max_depth=j)
      lr.fit(xtr,ytr)
      score = roc_auc_score(yvl,lr.predict(xvl))
      print('ROC AUC score:',score)
      cv_score.append(score)   
      print('Test Accuracy: ',accuracy_score(yvl,lr.predict(xvl)))
      print('Train Accuracy: ',accuracy_score(ytr,lr.predict(xtr)))
      print(confusion_matrix(yvl,lr.predict(xvl)))
      print(classification_report(yvl,lr.predict(xvl)))
      print('------------------------------------------------------------------')
      print('Validation Accuracy ',accuracy_score(y_test,lr.predict(X_test)))
      print(confusion_matrix(y_test,lr.predict(X_test)))
      print(classification_report(y_test,lr.predict(X_test)))
      print('------------------------------------------------------------------')
      i+=1

1 and 1 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC score: 0.6970071920191787
Test Accuracy:  0.6970071920191787
Train Accuracy:  0.6949433531822752
[[18385  7477]
 [ 8195 17667]]
              precision    recall  f1-score   support

           0       0.69      0.71      0.70     25862
           1       0.70      0.68      0.69     25862

   micro avg       0.70      0.70      0.70     51724
   macro avg       0.70      0.70      0.70     51724
weighted avg       0.70      0.70      0.70     51724

------------------------------------------------------------------
Validation Accuracy  0.7073833333333334
[[39383 16052]
 [ 1505  3060]]
              precision    recall  f1-score   support

           0       0.96      0.71      0.82     55435
           1       0.16      0.67      0.26      4565

   micro avg       0.71      0.71      0.71     60000
   macro avg       0.56      0.69      0.54     60000
weighted avg       0.90      0.71      0.78     60000

------------------------------------------------------------------
2 and

/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC score: 0.6567744180651149
Test Accuracy:  0.6567744180651148
Train Accuracy:  0.6594617585646895
[[20299  5563]
 [12190 13672]]
              precision    recall  f1-score   support

           0       0.62      0.78      0.70     25862
           1       0.71      0.53      0.61     25862

   micro avg       0.66      0.66      0.66     51724
   macro avg       0.67      0.66      0.65     51724
weighted avg       0.67      0.66      0.65     51724

------------------------------------------------------------------
Validation Accuracy  0.76675
[[43595 11840]
 [ 2155  2410]]
              precision    recall  f1-score   support

           0       0.95      0.79      0.86     55435
           1       0.17      0.53      0.26      4565

   micro avg       0.77      0.77      0.77     60000
   macro avg       0.56      0.66      0.56     60000
weighted avg       0.89      0.77      0.82     60000

------------------------------------------------------------------
3 and 1 of KFold

...

ROC AUC score: 0.9553785476761272
Test Accuracy:  0.9553785476761272
Train Accuracy:  0.969578918877117
[[24723  1139]
 [ 1169 24693]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     25862
           1       0.96      0.95      0.96     25862

   micro avg       0.96      0.96      0.96     51724
   macro avg       0.96      0.96      0.96     51724
weighted avg       0.96      0.96      0.96     51724

------------------------------------------------------------------
Validation Accuracy  0.8946
[[53108  2327]
 [ 3997   568]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     55435
           1       0.20      0.12      0.15      4565

   micro avg       0.89      0.89      0.89     60000
   macro avg       0.56      0.54      0.55     60000
weighted avg       0.87      0.89      0.88     60000

------------------------------------------------------------------
49 and 46 of KFold

/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC score: 0.953851210269894
Test Accuracy:  0.9538512102698941
Train Accuracy:  0.9698737529966747
[[24748  1114]
 [ 1273 24589]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     25862
           1       0.96      0.95      0.95     25862

   micro avg       0.95      0.95      0.95     51724
   macro avg       0.95      0.95      0.95     51724
weighted avg       0.95      0.95      0.95     51724

------------------------------------------------------------------
Validation Accuracy  0.8947666666666667
[[53130  2305]
 [ 4009   556]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     55435
           1       0.19      0.12      0.15      4565

   micro avg       0.89      0.89      0.89     60000
   macro avg       0.56      0.54      0.55     60000
weighted avg       0.87      0.89      0.88     60000

------------------------------------------------------------------
50 and

/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC score: 0.9547985461294564
Test Accuracy:  0.9547985461294564
Train Accuracy:  0.9698544196117856
[[24781  1081]
 [ 1257 24605]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     25862
           1       0.96      0.95      0.95     25862

   micro avg       0.95      0.95      0.95     51724
   macro avg       0.95      0.95      0.95     51724
weighted avg       0.95      0.95      0.95     51724

------------------------------------------------------------------
Validation Accuracy  0.8960166666666667
[[53238  2197]
 [ 4042   523]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     55435
           1       0.19      0.11      0.14      4565

   micro avg       0.90      0.90      0.90     60000
   macro avg       0.56      0.54      0.54     60000
weighted avg       0.87      0.90      0.88     60000

------------------------------------------------------------------


In [ ]:
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
for j in np.arange(1,50,5):
  for train_index,test_index in kf.split(X_train,y_train):
      print('{} and {} of KFold {}'.format(i,j,kf.n_splits))
      xtr,xvl = X_train.iloc[train_index],X_train.iloc[test_index]
      ytr,yvl = y_train.iloc[train_index],y_train.iloc[test_index]

      #model
      lr = RandomForestClassifier(max_depth=j)
      lr.fit(xtr,ytr)
      score = roc_auc_score(yvl,lr.predict(xvl))
      print('ROC AUC score:',score)
      cv_score.append(score)   
      print('Test Accuracy: ',accuracy_score(yvl,lr.predict(xvl)))
      print('Train Accuracy: ',accuracy_score(ytr,lr.predict(xtr)))
      print(confusion_matrix(yvl,lr.predict(xvl)))
      print(classification_report(yvl,lr.predict(xvl)))
      print('------------------------------------------------------------------')
      print('Validation Accuracy ',accuracy_score(y_test,lr.predict(X_test)))
      print(confusion_matrix(y_test,lr.predict(X_test)))
      print(classification_report(y_test,lr.predict(X_test)))
      print('------------------------------------------------------------------')
      i+=1

In [50]:
from imblearn.under_sampling import RandomUnderSampler
ros = RandomOverSampler()
X_undersampled, y_undersampled = ros.fit_sample(X_train, y_train)

In [51]:
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
for j in np.arange(1,50,5):
  for train_index,test_index in kf.split(X_undersampled,y_undersampled):
      print('{} and {} of KFold {}'.format(i,j,kf.n_splits))
      xtr,xvl = X_undersampled[train_index],X_undersampled[test_index]
      ytr,yvl = y_undersampled[train_index],y_undersampled[test_index]

      #model
      lr = RandomForestClassifier(max_depth=j)
      lr.fit(xtr,ytr)
      score = roc_auc_score(yvl,lr.predict(xvl))
      print('ROC AUC score:',score)
      cv_score.append(score)   
      print('Test Accuracy: ',accuracy_score(yvl,lr.predict(xvl)))
      print('Train Accuracy: ',accuracy_score(ytr,lr.predict(xtr)))
      print(confusion_matrix(yvl,lr.predict(xvl)))
      print(classification_report(yvl,lr.predict(xvl)))
      print('------------------------------------------------------------------')
      print('Validation Accuracy ',accuracy_score(y_test,lr.predict(X_test)))
      print(confusion_matrix(y_test,lr.predict(X_test)))
      print(classification_report(y_test,lr.predict(X_test)))
      print('------------------------------------------------------------------')
      i+=1

1 and 1 of KFold 5


/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC score: 0.6593264248704663
Test Accuracy:  0.6593264248704663
Train Accuracy:  0.6607957621220323
[[19621  6241]
 [11380 14482]]
              precision    recall  f1-score   support

           0       0.63      0.76      0.69     25862
           1       0.70      0.56      0.62     25862

   micro avg       0.66      0.66      0.66     51724
   macro avg       0.67      0.66      0.66     51724
weighted avg       0.67      0.66      0.66     51724

------------------------------------------------------------------
Validation Accuracy  0.7458166666666667
[[42205 13230]
 [ 2021  2544]]
              precision    recall  f1-score   support

           0       0.95      0.76      0.85     55435
           1       0.16      0.56      0.25      4565

   micro avg       0.75      0.75      0.75     60000
   macro avg       0.56      0.66      0.55     60000
weighted avg       0.89      0.75      0.80     60000

------------------------------------------------------------------
2 and

/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC score: 0.6869924986466631
Test Accuracy:  0.686992498646663
Train Accuracy:  0.6890901709071224
[[18413  7449]
 [ 8741 17121]]
              precision    recall  f1-score   support

           0       0.68      0.71      0.69     25862
           1       0.70      0.66      0.68     25862

   micro avg       0.69      0.69      0.69     51724
   macro avg       0.69      0.69      0.69     51724
weighted avg       0.69      0.69      0.69     51724

------------------------------------------------------------------
Validation Accuracy  0.7105833333333333
[[39624 15811]
 [ 1554  3011]]
              precision    recall  f1-score   support

           0       0.96      0.71      0.82     55435
           1       0.16      0.66      0.26      4565

   micro avg       0.71      0.71      0.71     60000
   macro avg       0.56      0.69      0.54     60000
weighted avg       0.90      0.71      0.78     60000

------------------------------------------------------------------
3 and 

...

ROC AUC score: 0.9563065501508005
Test Accuracy:  0.9563065501508003
Train Accuracy:  0.9695257520686722
[[24820  1042]
 [ 1218 24644]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     25862
           1       0.96      0.95      0.96     25862

   micro avg       0.96      0.96      0.96     51724
   macro avg       0.96      0.96      0.96     51724
weighted avg       0.96      0.96      0.96     51724

------------------------------------------------------------------
Validation Accuracy  0.8958166666666667
[[53246  2189]
 [ 4062   503]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     55435
           1       0.19      0.11      0.14      4565

   micro avg       0.90      0.90      0.90     60000
   macro avg       0.56      0.54      0.54     60000
weighted avg       0.87      0.90      0.88     60000

------------------------------------------------------------------
49 an

/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC score: 0.9549725465934576
Test Accuracy:  0.9549725465934575
Train Accuracy:  0.9698640863042302
[[24787  1075]
 [ 1254 24608]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     25862
           1       0.96      0.95      0.95     25862

   micro avg       0.95      0.95      0.95     51724
   macro avg       0.95      0.95      0.95     51724
weighted avg       0.95      0.95      0.95     51724

------------------------------------------------------------------
Validation Accuracy  0.8970666666666667
[[53296  2139]
 [ 4037   528]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.95     55435
           1       0.20      0.12      0.15      4565

   micro avg       0.90      0.90      0.90     60000
   macro avg       0.56      0.54      0.55     60000
weighted avg       0.87      0.90      0.88     60000

------------------------------------------------------------------
50 an

/Users/vaibhavgupta/anaconda3/envs/gatom-ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC score: 0.9564418838450236
Test Accuracy:  0.9564418838450236
Train Accuracy:  0.9695692521846725
[[24844  1018]
 [ 1235 24627]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     25862
           1       0.96      0.95      0.96     25862

   micro avg       0.96      0.96      0.96     51724
   macro avg       0.96      0.96      0.96     51724
weighted avg       0.96      0.96      0.96     51724

------------------------------------------------------------------
Validation Accuracy  0.8957333333333334
[[53290  2145]
 [ 4111   454]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     55435
           1       0.17      0.10      0.13      4565

   micro avg       0.90      0.90      0.90     60000
   macro avg       0.55      0.53      0.54     60000
weighted avg       0.87      0.90      0.88     60000

------------------------------------------------------------------


In [39]:
from sklearn.ensemble import AdaBoostClassifier

In [40]:
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
for j in np.arange(1,50,5):
  for train_index,test_index in kf.split(X_resampled,y_resampled):
      print('{} and {} of KFold {}'.format(i,j,kf.n_splits))
      xtr,xvl = X_resampled[train_index],X_resampled[test_index]
      ytr,yvl = y_resampled[train_index],y_resampled[test_index]

      #model
      ada = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1,
                         random_state=0)
      ada.fit(xtr,ytr)
      score = roc_auc_score(yvl,ada.predict(xvl))
      print('ROC AUC score:',score)
      cv_score.append(score)   
      print('Test Accuracy: ',accuracy_score(yvl,ada.predict(xvl)))
      print('Train Accuracy: ',accuracy_score(ytr,ada.predict(xtr)))
      print(confusion_matrix(yvl,ada.predict(xvl)))
      print(classification_report(yvl,ada.predict(xvl)))
      print('------------------------------------------------------------------')
      print('Validation Accuracy ',accuracy_score(y_test,ada.predict(X_test)))
      print(confusion_matrix(y_test,ada.predict(X_test)))
      print(classification_report(y_test,ada.predict(X_test)))
      print('------------------------------------------------------------------')
      i+=1

1 and 1 of KFold 5
ROC AUC score: 0.7473126595004255
Test Accuracy:  0.7473126595004254
Train Accuracy:  0.7419524785399427
[[18558  7304]
 [ 5766 20096]]
              precision    recall  f1-score   support

           0       0.76      0.72      0.74     25862
           1       0.73      0.78      0.75     25862

   micro avg       0.75      0.75      0.75     51724
   macro avg       0.75      0.75      0.75     51724
weighted avg       0.75      0.75      0.75     51724

------------------------------------------------------------------
Validation Accuracy  0.7187
[[39660 15775]
 [ 1103  3462]]
              precision    recall  f1-score   support

           0       0.97      0.72      0.82     55435
           1       0.18      0.76      0.29      4565

   micro avg       0.72      0.72      0.72     60000
   macro avg       0.58      0.74      0.56     60000
weighted avg       0.91      0.72      0.78     60000

-----------------------------------------------------------------

Validation Accuracy  0.7215166666666667
[[39867 15568]
 [ 1141  3424]]
              precision    recall  f1-score   support

           0       0.97      0.72      0.83     55435
           1       0.18      0.75      0.29      4565

   micro avg       0.72      0.72      0.72     60000
   macro avg       0.58      0.73      0.56     60000
weighted avg       0.91      0.72      0.79     60000

------------------------------------------------------------------
10 and 6 of KFold 5
ROC AUC score: 0.7402173072461526
Test Accuracy:  0.7402173072461526
Train Accuracy:  0.7421941458510556
[[18492  7370]
 [ 6067 19795]]
              precision    recall  f1-score   support

           0       0.75      0.72      0.73     25862
           1       0.73      0.77      0.75     25862

   micro avg       0.74      0.74      0.74     51724
   macro avg       0.74      0.74      0.74     51724
weighted avg       0.74      0.74      0.74     51724

----------------------------------------------------

ROC AUC score: 0.742595313587503
Test Accuracy:  0.7425953135875029
Train Accuracy:  0.7424019797386127
[[18547  7315]
 [ 5999 19863]]
              precision    recall  f1-score   support

           0       0.76      0.72      0.74     25862
           1       0.73      0.77      0.75     25862

   micro avg       0.74      0.74      0.74     51724
   macro avg       0.74      0.74      0.74     51724
weighted avg       0.74      0.74      0.74     51724

------------------------------------------------------------------
Validation Accuracy  0.71925
[[39696 15739]
 [ 1106  3459]]
              precision    recall  f1-score   support

           0       0.97      0.72      0.82     55435
           1       0.18      0.76      0.29      4565

   micro avg       0.72      0.72      0.72     60000
   macro avg       0.58      0.74      0.56     60000
weighted avg       0.91      0.72      0.78     60000

------------------------------------------------------------------
19 and 16 of KFol

              precision    recall  f1-score   support

           0       0.76      0.72      0.74     25862
           1       0.73      0.78      0.75     25862

   micro avg       0.75      0.75      0.75     51724
   macro avg       0.75      0.75      0.75     51724
weighted avg       0.75      0.75      0.75     51724

------------------------------------------------------------------
Validation Accuracy  0.7187
[[39660 15775]
 [ 1103  3462]]
              precision    recall  f1-score   support

           0       0.97      0.72      0.82     55435
           1       0.18      0.76      0.29      4565

   micro avg       0.72      0.72      0.72     60000
   macro avg       0.58      0.74      0.56     60000
weighted avg       0.91      0.72      0.78     60000

------------------------------------------------------------------
27 and 26 of KFold 5
ROC AUC score: 0.741937978501276
Test Accuracy:  0.741937978501276
Train Accuracy:  0.7419718119248317
[[18424  7438]
 [ 5910 19952]

Validation Accuracy  0.7215166666666667
[[39867 15568]
 [ 1141  3424]]
              precision    recall  f1-score   support

           0       0.97      0.72      0.83     55435
           1       0.18      0.75      0.29      4565

   micro avg       0.72      0.72      0.72     60000
   macro avg       0.58      0.73      0.56     60000
weighted avg       0.91      0.72      0.79     60000

------------------------------------------------------------------
35 and 31 of KFold 5
ROC AUC score: 0.7402173072461526
Test Accuracy:  0.7402173072461526
Train Accuracy:  0.7421941458510556
[[18492  7370]
 [ 6067 19795]]
              precision    recall  f1-score   support

           0       0.75      0.72      0.73     25862
           1       0.73      0.77      0.75     25862

   micro avg       0.74      0.74      0.74     51724
   macro avg       0.74      0.74      0.74     51724
weighted avg       0.74      0.74      0.74     51724

---------------------------------------------------

ROC AUC score: 0.742595313587503
Test Accuracy:  0.7425953135875029
Train Accuracy:  0.7424019797386127
[[18547  7315]
 [ 5999 19863]]
              precision    recall  f1-score   support

           0       0.76      0.72      0.74     25862
           1       0.73      0.77      0.75     25862

   micro avg       0.74      0.74      0.74     51724
   macro avg       0.74      0.74      0.74     51724
weighted avg       0.74      0.74      0.74     51724

------------------------------------------------------------------
Validation Accuracy  0.71925
[[39696 15739]
 [ 1106  3459]]
              precision    recall  f1-score   support

           0       0.97      0.72      0.82     55435
           1       0.18      0.76      0.29      4565

   micro avg       0.72      0.72      0.72     60000
   macro avg       0.58      0.74      0.56     60000
weighted avg       0.91      0.72      0.78     60000

------------------------------------------------------------------
44 and 41 of KFol

In [42]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [53]:
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
for j in np.arange(1,50,1):
  for train_index,test_index in kf.split(X_resampled,y_resampled):
      print('{} and {} of KFold {}'.format(i,j,kf.n_splits))
      xtr,xvl = X_resampled[train_index],X_resampled[test_index]
      ytr,yvl = y_resampled[train_index],y_resampled[test_index]

      #model
      xgb_model = XGBClassifier(objective="reg:linear", random_state=42)
      xgb_model.fit(xtr,ytr)
      score = roc_auc_score(yvl,xgb_model.predict(xvl))
      print('ROC AUC score:',score)
      cv_score.append(score)   
      print('Test Accuracy: ',accuracy_score(yvl,xgb_model.predict(xvl)))
      print('Train Accuracy: ',accuracy_score(ytr,xgb_model.predict(xtr)))
      i+=1

1 and 1 of KFold 5
ROC AUC score: 0.7481439950506534
Test Accuracy:  0.7481439950506534
Train Accuracy:  0.7463749903333076
2 and 1 of KFold 5
ROC AUC score: 0.7471773258062022
Test Accuracy:  0.7471773258062021
Train Accuracy:  0.7467664913773103
3 and 1 of KFold 5
ROC AUC score: 0.7473319928853144
Test Accuracy:  0.7473319928853144
Train Accuracy:  0.7482309952826541
4 and 1 of KFold 5
ROC AUC score: 0.744180651148403
Test Accuracy:  0.744180651148403
Train Accuracy:  0.7476606604284278
5 and 1 of KFold 5
ROC AUC score: 0.7437939834506226
Test Accuracy:  0.7437939834506225
Train Accuracy:  0.7465586574897533
6 and 2 of KFold 5
ROC AUC score: 0.7481439950506534
Test Accuracy:  0.7481439950506534
Train Accuracy:  0.7463749903333076
7 and 2 of KFold 5
ROC AUC score: 0.7471773258062022
Test Accuracy:  0.7471773258062021
Train Accuracy:  0.7467664913773103
8 and 2 of KFold 5
ROC AUC score: 0.7473319928853144
Test Accuracy:  0.7473319928853144
Train Accuracy:  0.7482309952826541
9 and 2 of

KeyboardInterrupt: 